In [2]:
from random import randint, choice

In [3]:
# Hit Points: 71
# Damage: 10

In [21]:
class duel(object):
    def __init__(self, A, B, handicap = 0):
        self.A = A
        self.B = B
        self.handicap = handicap
        
    def check(self):        
        if self.A.hp <= 0 or self.A.mana < 0:
                return self.B
        if self.B.hp <= 0 or self.B.mana < 0:
                return self.A
        return None
        
    def start(self):
        fight.history = []
        cont_spells = []
        step = 0
        while True:
            #print "step %d " % (step)
            #if len(cont_spells) == 0:
            #    spell = self.A.cast(dum, target = self.B)
            #    active_spells.append(spell)
            
            if not step % 2:
                self.A.hp -= self.handicap
                if self.check():
                    return self.check()
                
            for s in cont_spells[:]:
                if not s.tick():
                    s.end_effect()
                    cont_spells.remove(s)                                        
            
            if self.check():
                return self.check()            
            
            if not step % 2:
                active = self.A
                target = self.B
            else:
                active = self.B
                target = self.A
            
            if active.damage:
                #print "hit" #, "- active: ", [c.__class__.__name__ for c in cont_spells]
                target.hp -= max(1, active.damage - target.armor)
            
            if active.spells:
                spell = active.choose(ongoing = cont_spells, target = target, handicap = self.handicap, step = step)
                cont_spells.append(active.cast(spell, target = target))
                fight.history.append(spell)
            
                #print "player: hp %d | mana %d | mana spent %d | armor %d " % (self.A.hp, self.A.mana, self.A.mana_spent, self.A.armor)
                #print "boss:   hp %d |" % (self.B.hp)            
                #print "active: ", [c.__class__.__name__ for c in cont_spells]
            
            if self.check():
                return self.check()
            
            step += 1
            if step > 100:
                break

In [22]:
class character(object):
    def __init__(self, hp = 0, damage = 0, armor = 0, mana = 0, random = False):
        self.init_hp = hp        
        self.init_damage = damage
        self.init_armor = armor
        self.init_mana = mana
        self.spells = []
        self.random = random
        self.reset()
    
    def reset(self):
        self.hp = self.init_hp
        self.damage = self.init_damage
        self.armor = self.init_armor
        self.mana = self.init_mana
        self.mana_spent = 0
        self.equipment = []
        
    def equip(self, eq):
        cost = 0
        for e in eq:
            self.equipment.append(e[0])
            cost += e[1]
            self.damage += e[2]
            self.armor += e[3]
        return cost
    
    def choose(self, ongoing = [], target = None, handicap = 0, step = 0):        
        dot = 0
        for s in ongoing:
            if hasattr(s, 'dot'):
                dot += s.dot
        
        possible = [s for s in self.spells if s not in ongoing]
        if self.random:
            return choice(possible)
        
        if handicap and step > 18:
            return choice(possible)
        
        #if self.hp == 3:
        #    return possible[-1]
        
        if handicap:
            for p in possible:
                if type(p) == Magic_Missile and (p.damage + dot) >= target.hp:
                    return p
        else:
            for p in possible:
                if type(p) == Magic_Missile and (2*p.damage + 2*dot) >= target.hp:
                    return p
        
        if handicap:
            for p in possible:
                if type(p) == Recharge and self.mana < possible[0].manacost + p.manacost and target.hp > 23:
                    return p
        else:            
            for p in possible:
                if type(p) == Recharge and self.mana < possible[0].manacost + p.manacost and target.hp > 23:
                    return p
        return possible[0]
        #return choice(possible)            
            
    def cast(self, spell, target = None):
        spell.cast(caster = self, target = target)
        return spell
    
    def hit(self, enemy):
        enemy.hp -= max(1, self.damage - enemy.armor)
    
    def fight(self, enemy):
        while self.hp > 0 and enemy.hp > 0:
            self.hit(enemy)
            if enemy.hp <= 0:
                return True
            enemy.hit(self)
            if self.hp <= 0:
                return False
            #print self.hp, boss.hp

In [26]:
class spell(object):
    def __init__(self, duration = 0, manacost = 0):        
        self.duration = duration
        self.manacost = manacost
        #print "spell init", manacost
        
    def cast(self, caster = None, target = None):
        self.caster = caster
        self.target = target
        self.timer = self.duration
        caster.mana -= self.manacost
        caster.mana_spent += self.manacost
        self.cast_effect()
        #print "cast: ", self.__class__.__name__, " for ", self.manacost, " mana"
        return self
        
    def tick(self):
        self.timer -= 1
        self.cont_effect()
        if self.timer <= 0:
            return False
        return True
        
    def cast_effect(self):
        pass
    
    def cont_effect(self):
        pass
        
    def end_effect(self):
        pass
    
    def __repr__(self):
        return self.__class__.__name__

class Magic_Missile(spell):
    def __init__(self, damage = 4):
        self.damage = damage
        super(Magic_Missile, self).__init__(duration = 0, manacost = 53)
        
    def cast_effect(self):
        self.target.hp -= self.damage
        
class Drain(spell):
    def __init__(self, damage = 2):
        self.damage = damage
        super(Drain, self).__init__(duration = 0, manacost = 73)
        
    def cast_effect(self):
        self.target.hp -= self.damage
        self.caster.hp += self.damage
        
class Shield(spell):
    def __init__(self, effect = 7):
        self.effect = effect
        super(Shield, self).__init__(duration = 6, manacost = 113)
        
    def cast_effect(self):        
        self.caster.armor += self.effect
        
    def end_effect(self):
        self.caster.armor -= self.effect

class Poison(spell):
    def __init__(self, dot = 3):
        self.dot = dot
        super(Poison, self).__init__(duration = 6, manacost = 173)
        
    def cont_effect(self):
        self.target.hp -= self.dot
        
class Recharge(spell):
    def __init__(self, effect = 101):
        self.effect = effect
        super(Recharge, self).__init__(duration = 5, manacost = 229)
        
    def cont_effect(self):
        self.caster.mana += self.effect


In [27]:
player = character(hp = 50, mana = 500)
boss = character(hp = 71, damage = 10)
mm = Magic_Missile()
drain = Drain()
shield = Shield()
poison = Poison()
recharge = Recharge()
#player.spells = [shield, recharge, poison, mm, drain]
#player.spells = [shield, poison, recharge, mm, drain]
player.spells = [poison, shield, recharge, mm , drain]
# dum = dot()

In [28]:
fight = duel(player, boss)
player.reset()
boss.reset()
winner = fight.start()

In [29]:
print winner == player, player.mana_spent, player.hp, player.mana

True 1824 3 191


In [30]:
fight = duel(player, boss, handicap = 1)
#player.spells = [shield, recharge, poison, mm, drain]
#player.spells = [shield, drain, poison, recharge, mm]
#player.spells = [poison, shield, recharge, mm, drain]
player.spells = [shield, poison, recharge, mm] #, drain]
player.reset()
boss.reset()
winner = fight.start()

In [31]:
print winner == player, player.mana_spent, player.hp, boss.hp, player.mana

False 2053 -2 13 467


In [32]:
fight = duel(player, boss, handicap = 1)
i = 0
while True:
    player.reset()
    boss.reset()
    if fight.start() == player:
        break
    i += 1
    if not i % 100:
        print i, len(fight.history)
    #break
print i, player.mana_spent, player.hp, boss.hp, player.mana, fight.history

35 1937 1 0 78 [Shield, Recharge, Poison, Shield, Recharge, Poison, Shield, Recharge, Poison, Shield, Magic_Missile, Poison, Magic_Missile]
